In [79]:
import tweepy
import json
import pandas as pd
import spacy
import string
import joblib
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import English
from nltk.stem.porter import PorterStemmer
spacy.load('en')
parser = English()

### Twitter API Auth

In [80]:
consumer_key= 'XXXX'
consumer_secret= 'XXXX'
access_token= 'XXXX'
access_token_secret= 'XXXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Fetch Tweets

In [81]:
trackingList1 = ['India', 'modi', 'BJP']
trackingList2 = ['sports', 'cricket']
trackList = trackingList2
tweet_count = 0
n_tweets = 4
tweets_text = []


class StdOutListener(tweepy.StreamListener):
    def on_data(self, status):
        global tweet_count
        global n_tweets
        global stream
        if tweet_count < n_tweets:
            if json.loads(status)['lang'] == "en":
                print(json.loads(status)['text'])
                tweets_text.append(json.loads(status)['text'])
                tweet_count += 1
                return True
        else:
            stream.disconnect()

    def on_error(self, status):
        print(status)


stream = tweepy.Stream(auth, StdOutListener())
stream.filter(track=tracklist)

RT @ekramwaris: This govt is trying hard to destroy our country like its predecessor Hindu Mahasabha. It can't protect the borders but unle…
RT @bainjal: Instructions have gone out from the PMO that the farmers agitation should be blanked out from the media. Typical Modi headline…
RT @ProgIntl: LIVE, AT THE HOUR: 

Millions of workers and farmers are striking in India.

Why are they on strike? What are their demands?…
@UnemploymentPUA Obviously no party here is putting AMERICAN LIVES FIRST. FAMILIES AND CHILDREN MUST COME AHEAD OF… https://t.co/yajnzEIdGX


In [82]:
tweets_text

["RT @ekramwaris: This govt is trying hard to destroy our country like its predecessor Hindu Mahasabha. It can't protect the borders but unle…",
 'RT @bainjal: Instructions have gone out from the PMO that the farmers agitation should be blanked out from the media. Typical Modi headline…',
 'RT @ProgIntl: LIVE, AT THE HOUR: \n\nMillions of workers and farmers are striking in India.\n\nWhy are they on strike? What are their demands?…',
 '@UnemploymentPUA Obviously no party here is putting AMERICAN LIVES FIRST. FAMILIES AND CHILDREN MUST COME AHEAD OF… https://t.co/yajnzEIdGX']

### Pre Process Retrieved Tweets

In [83]:
STOPWORDS = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS)) 
SYMBOLCHARS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","''"]

def tokenizeText(textData):

    textData = textData.strip().replace("\n", " ").replace("\r", " ")
    textData = textData.lower()
    tokens = parser(textData)

    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # Remove Stop Words
    tokens = [tok for tok in tokens if tok.lower() not in STOPWORDS]
    # Remove Symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLCHARS]
    # Remove words with less than 3 characters
    tokens = [tok for tok in tokens if len(tok) >= 3]
    # Remove Non-Alphabetic Characters
    tokens = [tok for tok in tokens if tok.isalpha()]
    
    tokens = list(set(tokens))
    textData = ' '.join(tokens[:])
    return textData

In [84]:
tweets_text = pd.Series(tweets_text)
tweets_text = tweets_text.apply(lambda x:tokenizeText(x))
tweets_text

0    mahasabha unle govt protect hard trying predec...
1    typical media instructions gone headline blank...
2    india millions striking hour farmers workers s...
3    putting lives families party obviously ahead a...
dtype: object

### Predict Political Tweets (Indian Context)

In [85]:
classifier = joblib.load('./Saved Model/LR_Pol.pkl')
predict = classifier.predict(tweets_text)
pd.Series(predict)

0       POL
1       POL
2       POL
3    NOTPOL
dtype: object